In [41]:
import pandas as pd
import glob

In [42]:
import os
print("cwd:", os.getcwd())
print("files in cwd:", os.listdir())


cwd: /Users/davidarista/Energy-Heatwave-Predictor
files in cwd: ['merged_uk_demand_halfhourly.csv', 'uk.ipynb', 'london_weather_data_1979_to_2023.csv', 'archive', 'Untitled.ipynb', 'dataset.ipynb', 'london_energy.csv', 'dataset.py', '.gitignore', 'london-weather-data-from-1979-to-2023.zip', '.ipynb_checkpoints', '.git', 'london-homes-energy-data.zip', 'untitled.py']


In [43]:
import os
print("archive contains:", os.listdir("archive"))


archive contains: ['historic_demand_year_2010.csv', 'historic_demand_year_2011.csv', 'historic_demand_2009_2024.csv', 'historic_demand_year_2013.csv', 'historic_demand_year_2012.csv', 'historic_demand_year_2016.csv', 'historic_demand_year_2017.csv', 'historic_demand_year_2015.csv', 'historic_demand_year_2014.csv', 'historic_demand_year_2019.csv', 'historic_demand_year_2024.csv', 'historic_demand_year_2018.csv', 'historic_demand_year_2023.csv', 'historic_demand_year_2022.csv', 'historic_demand_year_2020.csv', 'historic_demand_year_2009.csv', 'historic_demand_year_2021.csv', '.ipynb_checkpoints', 'historic_demand_2009_2024_noNaN.csv', 'rista']


In [44]:
import glob

files = glob.glob("archive/historic_demand_year_*.csv")
print(files)


['archive/historic_demand_year_2010.csv', 'archive/historic_demand_year_2011.csv', 'archive/historic_demand_year_2013.csv', 'archive/historic_demand_year_2012.csv', 'archive/historic_demand_year_2016.csv', 'archive/historic_demand_year_2017.csv', 'archive/historic_demand_year_2015.csv', 'archive/historic_demand_year_2014.csv', 'archive/historic_demand_year_2019.csv', 'archive/historic_demand_year_2024.csv', 'archive/historic_demand_year_2018.csv', 'archive/historic_demand_year_2023.csv', 'archive/historic_demand_year_2022.csv', 'archive/historic_demand_year_2020.csv', 'archive/historic_demand_year_2009.csv', 'archive/historic_demand_year_2021.csv']


In [45]:
import pandas as pd
import glob

# load the list you just confirmed
files = glob.glob("archive/historic_demand_year_*.csv")

# read each file and concatenate
demand_df = pd.concat([pd.read_csv(f) for f in files], 
                      ignore_index=True)

# sanity check
print(f"Loaded {len(files)} files → {len(demand_df)} total rows")
demand_df.head()


Loaded 16 files → 279264 total rows


,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW
0,01-JAN-2010,1,36453,37593,32391,906,1786,0,0,0,306,1977,0,0,-234,0,0,NaN,NaN
1,01-JAN-2010,2,36680,37739,32590,926,1786,0,0,0,223,2000,0,0,-236,0,0,NaN,NaN
2,01-JAN-2010,3,36343,37467,32291,926,1786,0,0,0,288,2000,0,0,-236,0,0,NaN,NaN
3,01-JAN-2010,4,35362,36489,31366,952,1786,0,0,0,291,2000,0,0,-236,0,0,NaN,NaN
4,01-JAN-2010,5,34366,35563,30522,952,1786,0,0,0,361,1999,0,0,-236,0,0,NaN,NaN


In [46]:
# Attempt parsing, coercing errors to NaT
parsed = pd.to_datetime(
    demand_df['SETTLEMENT_DATE'],
    format='%d-%b-%Y',
    errors='coerce'
)

# Mask of bad conversions
bad = parsed.isna()

# What unique bad values do we have?
print("Bad date strings:", demand_df.loc[bad, 'SETTLEMENT_DATE'].unique())

# Peek at their full rows
print(demand_df.loc[bad].head())


Bad date strings: [2804 2834 2868 ... 32976 32231 28829]
       SETTLEMENT_DATE  SETTLEMENT_PERIOD  ND    TSD  ENGLAND_WALES_DEMAND  \
157776            2804               6488   0  16726                     0   
157777            2834               6488   0  16726                     0   
157778            2868               6488   0  16726                     0   
157779            2901               6488   0  16726                     0   
157780            2933               6488   0  16726                     0   

        EMBEDDED_WIND_GENERATION  EMBEDDED_WIND_CAPACITY  \
157776                       808                     -71   
157777                       895                     288   
157778                      1441                     466   
157779                      1564                     262   
157780                      1454                     595   

        EMBEDDED_SOLAR_GENERATION  EMBEDDED_SOLAR_CAPACITY  NON_BM_STOR  \
157776                        417     

In [47]:
dates = demand_df['SETTLEMENT_DATE'].astype(str)


In [48]:
valid_mask = dates.str.match(r'^\d{2}-[A-Z]{3}-\d{4}$').fillna(False)


In [49]:
print("Dropping", (~valid_mask).sum(), "bad rows")
demand_df = demand_df.loc[valid_mask].copy()


Dropping 51360 bad rows


In [50]:
demand_df['datetime'] = (
    pd.to_datetime(demand_df['SETTLEMENT_DATE'], format='%d-%b-%Y')
    + pd.to_timedelta((demand_df['SETTLEMENT_PERIOD'] - 1) * 30, unit='m')
)

In [51]:
print(demand_df[['SETTLEMENT_DATE','SETTLEMENT_PERIOD','datetime']].head())


  SETTLEMENT_DATE  SETTLEMENT_PERIOD            datetime
0     01-JAN-2010                  1 2010-01-01 00:00:00
1     01-JAN-2010                  2 2010-01-01 00:30:00
2     01-JAN-2010                  3 2010-01-01 01:00:00
3     01-JAN-2010                  4 2010-01-01 01:30:00
4     01-JAN-2010                  5 2010-01-01 02:00:00


In [52]:
demand_df['date'] = demand_df['datetime'].dt.date
daily_demand = (
    demand_df
    .groupby('date')['ENGLAND_WALES_DEMAND']
    .sum()
    .reset_index()
)

In [53]:
daily_demand['date'] = pd.to_datetime(daily_demand['date'])


In [54]:
import pandas as pd
import glob

# 1. Grab every year-by-year file
files = glob.glob("archive/historic_demand_year_*.csv")

# 2. Read & concatenate them
demand_df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

# 3. Drop rows where SETTLEMENT_DATE isn’t in DD-MMM-YYYY form
mask = demand_df['SETTLEMENT_DATE'].astype(str).str.match(r'^\d{2}-[A-Z]{3}-\d{4}$').fillna(False)
demand_df = demand_df.loc[mask].copy()

# 4. (Optional) Parse to a datetime if you need it
demand_df['datetime'] = (
    pd.to_datetime(demand_df['SETTLEMENT_DATE'], format='%d-%b-%Y')
    + pd.to_timedelta((demand_df['SETTLEMENT_PERIOD'] - 1) * 30, unit='m')
)

# 5. Inspect the result
print("Total rows after merge:", len(demand_df))
demand_df.head()


Total rows after merge: 227904


,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,datetime
0,01-JAN-2010,1,36453,37593,32391,906,1786,0,0,0,306,1977,0,0,-234,0,0,NaN,NaN,2010-01-01 00:00:00
1,01-JAN-2010,2,36680,37739,32590,926,1786,0,0,0,223,2000,0,0,-236,0,0,NaN,NaN,2010-01-01 00:30:00
2,01-JAN-2010,3,36343,37467,32291,926,1786,0,0,0,288,2000,0,0,-236,0,0,NaN,NaN,2010-01-01 01:00:00
3,01-JAN-2010,4,35362,36489,31366,952,1786,0,0,0,291,2000,0,0,-236,0,0,NaN,NaN,2010-01-01 01:30:00
4,01-JAN-2010,5,34366,35563,30522,952,1786,0,0,0,361,1999,0,0,-236,0,0,NaN,NaN,2010-01-01 02:00:00


In [55]:
# Save the full half-hourly demand merge
demand_df.to_csv("merged_uk_demand_halfhourly.csv", index=False)

# If you did the daily aggregation & merge:

In [56]:
import os
print(os.listdir())


['merged_uk_demand_halfhourly.csv', 'uk.ipynb', 'london_weather_data_1979_to_2023.csv', 'archive', 'Untitled.ipynb', 'dataset.ipynb', 'london_energy.csv', 'dataset.py', '.gitignore', 'london-weather-data-from-1979-to-2023.zip', '.ipynb_checkpoints', '.git', 'london-homes-energy-data.zip', 'untitled.py']


In [59]:
# after you’ve built df (weather) and avg_kwh (energy), run:
# … earlier cells …
df = pd.read_csv("london_weather_data_1979_to_2023.csv")
df['TX'] = df['TX']/10
df['date'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

df2 = pd.read_csv("london_energy.csv")
avg_kwh = (df2
            .groupby('Date')['KWH']
            .mean()
            .reset_index(name='consumption'))
avg_kwh['date'] = pd.to_datetime(avg_kwh['Date'])

# ← RIGHT HERE, right after those two prep steps:
merged_df = pd.merge(
    avg_kwh,
    df.drop(columns=['DATE']),  # drop the raw DATE column
    on='date',
    how='inner'
)

# sanity-check
print("Merged rows:", len(merged_df))
merged_df.head()




merged_df = pd.merge(
    avg_kwh,
    df.drop(columns=['DATE']),
    on='date',
    how='inner'
)


Merged rows: 829


In [60]:
# 1. Aggregate half‐hourly → daily total demand
demand_df['date'] = demand_df['datetime'].dt.date
daily_demand = (
    demand_df
      .groupby('date')['ENGLAND_WALES_DEMAND']
      .sum()
      .reset_index()
)
daily_demand['date'] = pd.to_datetime(daily_demand['date'])

# 2. Merge with your London weather+energy df (merged_df) on 'date'
final_df = daily_demand.merge(
    merged_df,    # your existing London df with a 'date' column
    on='date',
    how='inner'
)

print("Rows after merge:", len(final_df))
final_df.head()

Rows after merge: 829


,date,ENGLAND_WALES_DEMAND,Date,consumption,TX,Q_TX,TN,Q_TN,TG,Q_TG,...,RR,Q_RR,QQ,Q_QQ,PP,Q_PP,HU,Q_HU,CC,Q_CC
0,2011-11-23,1726127,2011-11-23,6.952692,13.5,0,26.0,0,68.0,0,...,2.0,0,35.0,0,10272.0,0,93.0,0,7.0,0
1,2011-11-24,1686910,2011-11-24,8.536480,12.5,0,37.0,0,86.0,0,...,2.0,0,35.0,0,10271.0,0,88.0,0,3.0,0
2,2011-11-25,1657598,2011-11-25,9.499781,14.0,0,95.0,0,110.0,0,...,0.0,0,52.0,0,10245.0,0,78.0,0,3.0,0
3,2011-11-26,1497543,2011-11-26,10.267707,13.9,0,63.0,0,102.0,0,...,0.0,0,24.0,0,10258.0,0,79.0,0,4.0,0
4,2011-11-27,1436364,2011-11-27,10.850805,13.2,0,97.0,0,118.0,0,...,0.0,0,55.0,0,10213.0,0,69.0,0,3.0,0


In [61]:
print(len(avg_kwh))
print(avg_kwh['date'].min(), avg_kwh['date'].max())


829
2011-11-23 00:00:00 2014-02-28 00:00:00


In [62]:
print(daily_demand['date'].min(), daily_demand['date'].max())


2009-01-01 00:00:00 2021-12-31 00:00:00


In [63]:
# How many days of energy data do we have?
print("Energy days:", len(avg_kwh))
print("Energy date range:", avg_kwh['date'].min(), "→", avg_kwh['date'].max())

# And confirm the merged period in final_df
print("Merged days:", len(final_df))
print("Merged date range:", final_df['date'].min(), "→", final_df['date'].max())


Energy days: 829
Energy date range: 2011-11-23 00:00:00 → 2014-02-28 00:00:00
Merged days: 829
Merged date range: 2011-11-23 00:00:00 → 2014-02-28 00:00:00


In [64]:
final_df.to_csv("merged_daily_demand_weather_energy.csv", index=False)